In [9]:
! pip install -q tensorflow numpy
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./
! cp -r drive/My\ Drive/Colab\ Notebooks/models/aaaumberto_feedforward ./

In [2]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [3]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [4]:
from tensorflow import keras

model = keras.models.load_model('aaaumberto_feedforward')

In [10]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [11]:
import numpy as np

X, y = np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values

In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

ssc = StandardScaler()
mms = MinMaxScaler()

In [13]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ohe(create_map())
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = pickle.load(open('umberto_embeddings.pkl', 'rb'))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

(1600, 1)
(1600, 2049)
(1600, 1)
(1600, 2050)
(1600, 71)
(1600, 2121)
(1600, 768)
(1600, 2889)


In [41]:
test_loss, test_acc = model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 7ms/step - loss: 0.7137 - binary_accuracy: 0.8188


In [40]:
X_test = pickle.load(open('X_test.pkl', 'rb'))
y_test = pickle.load(open('y_test.pkl', 'rb'))

In [42]:
predictions = model.predict_classes(X_test, verbose=0)

In [46]:
fp = 0
tp = 0
fn = 0 
c = 0
incorrect = []

for pred, test in zip(predictions, y_test):
  if pred[0] != test[0]:
    incorrect.append(c)
  if pred[0] == 1:
    if pred[0] == test[0]:
      tp += 1
    else:
      fp += 1
  elif pred[0] != test[0]:
    fn += 1
  c += 1

print(tp, fp, fn)

145 44 14


In [26]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)

In [27]:
print('Precision: ', precision, '\nRecall: ', recall, '\nF1 Score: ', f1)

Precision:  0.7671957671957672 
Recall:  0.9119496855345912 
F1 Score:  0.8333333333333334


In [48]:
print(incorrect)
print(len(incorrect))

[0, 8, 11, 19, 21, 24, 40, 46, 47, 50, 52, 59, 60, 63, 67, 69, 80, 97, 101, 106, 113, 120, 123, 124, 125, 129, 138, 140, 149, 156, 164, 166, 168, 184, 191, 203, 209, 210, 214, 216, 218, 225, 228, 230, 237, 241, 254, 262, 271, 276, 281, 284, 291, 297, 304, 306, 310, 311]
58


In [69]:
X_inc = []

for i, val in enumerate(X):
  for j in incorrect:
    if np.all(X_test[j][:1000] == val[:1000]):
      X_inc.append(i)

In [71]:
print(X_inc)

[50, 52, 55, 68, 71, 75, 94, 125, 138, 145, 178, 203, 228, 232, 263, 280, 287, 292, 310, 332, 335, 347, 359, 402, 406, 419, 427, 469, 470, 478, 504, 510, 540, 609, 665, 670, 703, 712, 732, 746, 762, 775, 828, 844, 847, 956, 1003, 1094, 1130, 1209, 1238, 1351, 1371, 1391, 1505, 1559, 1591, 1598]
